In [14]:
import muda
import glob
import os
import tqdm

folder = "fold1"
jams_files_path = f"data/UrbanSound8K/jams-data/{folder}"
original_jams_file_path = f"{jams_files_path}/original/jams"

deformers_with_name = []

# Setup background deformer
background_noise_deformation_name = "bgnoise"
background_audio_files_glob = "data/UrbanSound8K/background_noise/*.wav"
audio_files_glob = f"data/UrbanSound8K/audio/{folder}/*.wav"
background_deformer = muda.deformers.BackgroundNoise(files=glob.glob(background_audio_files_glob))

deformers_with_name.append((background_noise_deformation_name, background_deformer))

# Setup time stretching deformer
time_stretch_deformation_name = "stretch"
time_stretch_deformer = muda.Union(steps=[(f"time-stretch-{i}", muda.deformers.TimeStretch(rate=stretch_factor)) for i, stretch_factor in enumerate([0.81, 0.93, 1.07, 1.23])])

deformers_with_name.append((time_stretch_deformation_name, time_stretch_deformer))

# Setup pitch shifting 1
pitch_shift_1_name = "pitch1"
pitch_shift_1_deformer = muda.Union(steps=[(f"pitch-shift-1-{i}", muda.deformers.PitchShift(n_semitones=semitone_shift_factor)) for i, semitone_shift_factor in enumerate([-2, -1, 1, 2])])

deformers_with_name.append((pitch_shift_1_name, pitch_shift_1_deformer))

# Setup pitch shifting 2
pitch_shift_2_name = "pitch2"
pitch_shift_2_deformer = muda.Union(steps=[(f"pitch-shift-2-{i}", muda.deformers.PitchShift(n_semitones=semitone_shift_factor)) for i, semitone_shift_factor in enumerate([-3.5, -2.5, 2.5, 3.5])])

deformers_with_name.append((pitch_shift_2_name, pitch_shift_2_deformer))

# Setup Dynamic Range compression
dynamic_range_compression_name = "drc"
dynamic_range_compression_deformer = muda.deformers.DynamicRangeCompression(preset=["music standard", "film standard", "speech", "radio"])

deformers_with_name.append((dynamic_range_compression_name, dynamic_range_compression_deformer))

audio_files_paths = sorted(glob.glob(audio_files_glob))
for deformer_name, deformer in deformers_with_name:
    print(f"Processing files with deformer: {deformer_name} ⚙️")

    for audio_file_path in tqdm.tqdm(audio_files_paths):
        deformed_audio_folder_path = f"{jams_files_path}/{deformer_name}/audio"
        deformed_jams_folder_path = f"{jams_files_path}/{deformer_name}/jams"
        os.makedirs(deformed_audio_folder_path, exist_ok=True)
        os.makedirs(deformed_jams_folder_path, exist_ok=True)
        file_name = os.path.basename(audio_file_path)[:-4]
        original_jam = muda.load_jam_audio(f"{original_jams_file_path}/{file_name}.jams", audio_file=audio_file_path)
        for i, deformed_jam in enumerate(deformer.transform(original_jam)):
            muda.save(f"{deformed_audio_folder_path}/{file_name}_{deformer_name}{i}.wav", f"{deformed_jams_folder_path}/{file_name}_{deformer_name}{i}.jams", deformed_jam)

    print(f"Done processing files with deformer: {deformer_name} ✅")

Processing files with deformer: drc ⚙️


  0%|          | 0/873 [00:00<?, ?it/s]sox WARN compand: compand clipped 9 samples; decrease volume?
sox WARN dither: dither clipped 7 samples; decrease volume?
  6%|▌         | 51/873 [00:06<01:35,  8.57it/s]sox WARN compand: compand clipped 5 samples; decrease volume?
sox WARN dither: dither clipped 5 samples; decrease volume?
  6%|▌         | 52/873 [00:07<01:35,  8.60it/s]sox WARN compand: compand clipped 2 samples; decrease volume?
sox WARN dither: dither clipped 2 samples; decrease volume?
  6%|▌         | 53/873 [00:07<01:35,  8.62it/s]sox WARN compand: compand clipped 1 samples; decrease volume?
sox WARN dither: dither clipped 1 samples; decrease volume?
  6%|▋         | 55/873 [00:07<01:35,  8.58it/s]sox WARN compand: compand clipped 3 samples; decrease volume?
sox WARN dither: dither clipped 3 samples; decrease volume?
  9%|▊         | 76/873 [00:09<01:31,  8.69it/s]sox WARN compand: compand clipped 5 samples; decrease volume?
sox WARN dither: dither clipped 5 samples; decrea

Done processing files with deformer: drc ✅
